In [1]:
import sys
# sys.path.insert(1, '/home/linus/projects/fdl/crisp')

In [2]:
import os
import pandas as pd
os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [3]:
from models.Deconfounder import *
from models.TorchMultiClassDeconfounder import *
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [21]:
from synthetic.facebook_synthetic_data_generator import generator_example

#n_example = 1
dim_inv=5
n_bin=0
dim_spu=10
n_exp_train = 1000
n_exp_test = 400
n_env=1
save_dir= 'data/synthetic'
test=False 

In [22]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "None", "multi-class", "real-valued"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID'],
        'synthetic_train_test_split' : True # use this to split in synthetic_generator
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud" : False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [23]:
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}

In [24]:
results = {}

for n_example in [2]: #[1,2,3,5]:
    config["data_options"]["synthetic_train_test_split"] = True
    output_data_regime = config["data_options"]["output_data_regime"][n_example-1]
    print("\n\nn_example:", n_example, "output_data_regime:", output_data_regime, "\n\n")
    generator_example(n_example, dim_inv, dim_spu, n_exp_train, n_env ,save_dir, test=False)#, n_bin) # generate pickle with train data
    generator_example(n_example, dim_inv, dim_spu, n_exp_test, n_env ,save_dir, test=True)  # generate pickle with test data
    outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_train, n_env,"False"))
    outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_test, n_env,"True"))
    
    if n_example == 6:
        outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s_n_bin_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,"False",n_bin))
        outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s_n_bin_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,"True",n_bin))
        
    df_train = pd.read_pickle(outfile_train)
    df_test = pd.read_pickle(outfile_test)

    config["data_options"]["dataset_fp_train"] = outfile_train
    config["data_options"]["dataset_fp_test"] = outfile_test
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])
    environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

    dcf_args["target"] = config["data_options"]["targets"]
    dcf_args["output_data_regime"] = output_data_regime
    dcf_args["columns"] = test_dataset.predictor_columns

    dcf = TorchMultiClassDeconfounder(environment_datasets, val_dataset, test_dataset, dcf_args)
    dcf_results_dict = dcf.predictor_results()
    
    results[n_example] = dcf_results_dict




n_example: 2 output_data_regime: binary 


Environments variables: {'E0': {'p': 0.95, 's': 0.3}}
Generated Synthetic Data according to the Facebook setup Example: 2
Environments variables: {'E0': {'p': 0.95, 's': 0.3}}
Generated Synthetic Data according to the Facebook setup Example: 2
Running a per sample experiment
Loaded  1  train environments
Env  0  has  800  samples
X shape  (800, 15)  y shape  (800, 1)
Loaded val set, X shape: (200, 15)  y shape:  (200, 1)
Loaded test set, X shape: (206, 15)  y shape:  (206, 1)
testshape: (206, 16)
valshape: (200, 16)
trainshape: (800, 16)
Number of features: 15
to drop: []
dropped 0 std columns of self.dfX
Number of features after correlated and 0 std features elimination: 6
Inferred W:
[[0.7105555  0.71261823 0.7523504  0.757472   0.73638386 0.7467199 ]]
Standard Deviation:
[[0.00432342 0.00618804 0.00530751 0.00560048 0.00559799 0.00530775]]
pval: 0.13906921241050121
predictions: (200, 1)
truth: (200,)
Acc/r_squared: 100.0
dcf_coefs: [[0.37

In [25]:
result = results.get(n_example).get("to_bucket")
result.keys()

dict_keys(['method', 'features', 'coefficients', 'pvals', 'test_acc', 'test_acc_std'])

In [15]:
result

{'method': 'Deconfounder',
 'features': None,
 'coefficients': None,
 'pvals': None,
 'test_acc': 0.0,
 'test_acc_std': 0.0}

In [11]:
if output_data_regime == "multi-class":
    r = sorted(zip(result.get("features"), np.abs(result.get("coefficients")).sum(axis=1)), key=lambda x: x[1], reverse=True)
else:
    r = sorted(zip(result.get("features"), result.get("coefficients")), key=lambda x: abs(x[1]), reverse=True)
r

TypeError: zip argument #1 must support iteration

In [20]:
#from synthetic.synthetic_generator import synthetic_generator
#df = synthetic_generator()
#df["Target"] = np.random.randint(0,3,size=df.shape[0])
#df.to_pickle("../data/test_multiclass.pkl")

In [24]:
import pandas as pd
import pickle
#with open("../data/test.pkl",'rb') as file:
with open("../data/test_multiclass.pkl",'rb') as file:
    df = pickle.load(file)

In [55]:
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

Running a per sample experiment
Loaded  2  train environments
Env  0  has  406  samples
X shape  (406, 4)  y shape  (406, 1)
Env  1  has  404  samples
X shape  (404, 4)  y shape  (404, 1)
Loaded val set, X shape: (90, 4)  y shape:  (90, 1)
Loaded test set, X shape: (100, 4)  y shape:  (100, 1)


In [56]:
# run DCF
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}
dcf_args["columns"] = test_dataset.predictor_columns

dcf = TorchMultiClassDeconfounder(environment_datasets, val_dataset, test_dataset, dcf_args)

dcf_results_dict = dcf.predictor_results()
print("Finished DCF")
print(dcf_results_dict)

testshape: (100, 5)
valshape: (90, 5)
trainshape: (810, 5)
Number of features: 4
to drop: []
dropped 0 std columns of self.dfX
Number of features after correlated and 0 std features elimination: 2
Inferred W:
[[0.84723186 0.8350464 ]]
Standard Deviation:
[[0.00468905 0.00620451]]
pval: 0.2171940298507463
IF TRIGGERED (1, 3)
predictions: [[0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]]
truth: [0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0.

In [57]:
dcf_results_dict["results"]["Features"]

,value,pval
Causal_1,0.635266,0.0
Non_causal_1,-0.271883,0.0
Z1,-0.020297,0.0


In [25]:
dcf_results_dict["results"]["Features"]

,value,value_1,value_2,pval,pval_1,pval_2
Non_causal_0,0.278975,-0.199761,-0.217077,0.0,0.0,0.0
Non_causal_1,0.018610,-0.189191,-0.017036,0.0,0.0,0.0
Non_causal_2,0.064372,0.103798,0.132453,0.0,0.0,0.0
Non_causal_3,0.005396,0.279146,0.219328,0.0,0.0,0.0
Non_causal_4,-0.205513,-0.113096,-0.229808,0.0,0.0,0.0
Non_causal_5,0.168901,-0.194200,0.055705,0.0,0.0,0.0
Causal_0,-0.130071,0.290094,0.183545,0.0,0.0,0.0
Non_causal_6,0.007791,-0.213810,0.196901,0.0,0.0,0.0
Causal_1,0.186078,0.030202,-0.106112,0.0,0.0,0.0
Non_causal_7,-0.032950,0.175299,0.102825,0.0,0.0,0.0


In [8]:
# run DCF
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}
dcf_args["columns"] = test_dataset.predictor_columns

dcf = Deconfounder(environment_datasets, val_dataset, test_dataset, dcf_args)

dcf_results_dict = dcf.predictor_results()
print("Finished DCF")
print(dcf_results_dict)

testshape: (17, 11)
valshape: (11, 11)
trainshape: (72, 11)
Number of features: 10
to drop: []
dropped 0 std columns of self.dfX
Number of features after correlated and 0 std features elimination: 10



/home/jupyter/crisp/models/Deconfounder.py:203: UserWarning: tfp.edward2 module is deprecated and will be removed on 2019-12-01. Use https://github.com/google/edward2 library instead.
  log_joint = ed.make_log_joint_fn(self.ppca_model)




Inferred W:
[[-0.10981557  0.1039127  -0.16744502  0.1487232   0.53235054 -0.28796935
   0.82654893  0.8187947  -0.98794115  0.6546716 ]]
Standard Deviation:
[[0.01360007 0.01331744 0.01420823 0.01452198 0.01411408 0.01400418
  0.01554985 0.01330545 0.01366075 0.01458096]]
pval: 0.01808988764044944
Inferred W:
[[-0.19818623  0.7873972   0.00285349  0.29324788  0.3645276   0.6014306
   0.5623061  -0.09629191 -0.7295748   0.8619132 ]]
Standard Deviation:
[[0.01415863 0.01535793 0.01560735 0.0138423  0.01492829 0.01394129
  0.01444643 0.01243034 0.01364147 0.0154509 ]]
pval: 0.03651685393258427
Inferred W:
[[-0.41313574  0.60376585 -0.39951774  0.16148852  0.14315881  0.6452538
   0.42083964 -0.23102091  0.01341048  0.33146822]]
Standard Deviation:
[[0.06144624 0.06346545 0.06012443 0.06185041 0.06525916 0.04017721
  0.05721071 0.05603929 0.06690963 0.05793456]]
pval: 0.10333333333333333
predictions: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
truth: [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Acc: 0.636

In [17]:
torch_ICP_results_dict['selected_features']

array(['Non_causal_0', 'Non_causal_1', 'Non_causal_2', 'Non_causal_3',
       'Non_causal_4', 'Non_causal_5', 'Non_causal_6', 'Causal_0',
       'Non_causal_7', 'Non_causal_8'], dtype='<U12')